In [1]:
data_root = '/home/scott/Documents/SPEC_Spider'

In [2]:
!tree /home/scott/Documents/SPEC_Spider

/home/scott/Documents/SPEC_Spider
├── cpu
│   ├── cpu2006
│   │   ├── SPECfp.csv
│   │   ├── SPECfp_rate.csv
│   │   ├── SPECint.csv
│   │   └── SPECint_rate.csv
│   └── cpu2017
│       ├── CFP2017_rate.csv
│       ├── CFP2017_speed.csv
│       ├── CINT2017_rate.csv
│       └── CINT2017_speed.csv
├── java
│   ├── jbb2015
│   │   ├── SPECjbb2015-Composite.csv
│   │   ├── SPECjbb2015-Distributed.csv
│   │   └── SPECjbb2015-MultiJVM.csv
│   └── jvm2008
│       └── jvm2008.csv
├── jbb2015.csv
└── power
    └── ssj2008.csv

7 directories, 14 files


In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
c2017_rfp = pd.read_csv(f"{data_root}/cpu/cpu2017/CFP2017_rate.csv")
c2017_sfp = pd.read_csv(f"{data_root}/cpu/cpu2017/CFP2017_speed.csv")
c2017_rint = pd.read_csv(f"{data_root}/cpu/cpu2017/CINT2017_rate.csv")
c2017_sint = pd.read_csv(f"{data_root}/cpu/cpu2017/CINT2017_speed.csv")

c2006_rfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp_rate.csv")
c2006_sfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp.csv")
c2006_rint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint_rate.csv")
c2006_sint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint.csv")

j2015_com = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-Composite.csv")
j2015_dis = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-Distributed.csv")
j2015_mul = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-MultiJVM.csv")

jvm2008 = pd.read_csv(f"{data_root}/java/jvm2008/jvm2008.csv")
ssj2008 = pd.read_csv(f"{data_root}/power/ssj2008.csv")

In [5]:
cpu2017_columns = [
    'Suite', 'Hardware Vendor', 'System Name', 'Baseline',
    'Test Date', 'HW Avail', 'CPU Name', 'Max MHz',
    'Nominal', 'Enabled', 'Orderable', 'L1',
    'L2', 'L3', 'Memory', 'Storage',
    'OS', 'File System', 'URL Suffix',
]
c2017_rfp = c2017_rfp[cpu2017_columns]
c2017_sfp = c2017_sfp[cpu2017_columns]
c2017_rint = c2017_rint[cpu2017_columns]
c2017_sint = c2017_sint[cpu2017_columns]

In [6]:
c2017 = pd.concat([c2017_rfp, c2017_sfp, c2017_rint, c2017_sint]).reset_index(drop=True)

In [7]:
c2017.columns

Index(['Suite', 'Hardware Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'Max MHz', 'Nominal', 'Enabled', 'Orderable',
       'L1', 'L2', 'L3', 'Memory', 'Storage', 'OS', 'File System',
       'URL Suffix'],
      dtype='object')

In [8]:
c2017_rename_dict = {
    'Hardware Vendor': 'HW Vendor',
    'Nominal': 'CPU MHz',
    'Enabled': 'CPU Enabled',
    'Orderable': 'CPU Orderable',
}
c2017.rename(columns=c2017_rename_dict, inplace=True)

In [9]:
c2017 = c2017[~c2017.isnull().any(axis=1)].reset_index(drop=True)

In [10]:
def get_cpu_vendor(cpu_name):
    item = cpu_name.split()[0]
    if item in ['Intel', 'AMD', 'Huawei']:
        vendor = item
    else:
        vendor = 'Other'
    return vendor
c2017['CPU Vendor'] = c2017['CPU Name'].apply(lambda x: get_cpu_vendor(x))

In [11]:
c2017['Max MHz'] = c2017['Max MHz'].apply(lambda x: round(x / 1000, 2))
c2017['CPU MHz'] = c2017['CPU MHz'].apply(lambda x: round(x / 1000, 2))

In [12]:
c2017['Max MHz'].unique()

array([3.5 , 4.  , 3.4 , 3.2 , 3.7 , 3.6 , 3.3 , 4.3 , 4.2 , 3.9 , 3.49,
       3.86, 3.8 , 5.1 , 3.65, 3.35, 5.  , 3.1 , 3.45, 3.67, 4.1 , 4.7 ,
       4.9 , 2.9 , 4.6 , 4.5 , 4.4 , 3.  , 2.7 , 1.7 , 2.8 , 2.6 , 4.13,
       2.1 , 1.9 , 4.8 , 4.35])

In [13]:
c2017['CPU MHz'].unique()

array([2.4 , 2.2 , 2.1 , 2.  , 2.9 , 3.6 , 2.8 , 2.3 , 2.6 , 2.7 , 2.5 ,
       3.  , 2.93, 3.33, 3.05, 3.2 , 3.1 , 2.75, 2.45, 4.  , 3.9 , 3.5 ,
       3.7 , 2.35, 3.4 , 3.8 , 2.25, 3.3 , 1.9 , 1.8 , 1.7 , 4.13, 2.65,
       2.95, 2.85, 4.1 , 1.3 , 4.25, 3.55])

In [14]:
c2017.columns

Index(['Suite', 'HW Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'Max MHz', 'CPU MHz', 'CPU Enabled',
       'CPU Orderable', 'L1', 'L2', 'L3', 'Memory', 'Storage', 'OS',
       'File System', 'URL Suffix', 'CPU Vendor'],
      dtype='object')

In [24]:
def parse_cpu_enabled(info):
    items = info.split(',')
    threads_per_core = 1
    total_cores = int(items[0].split()[0])
    chips = int(items[1].split()[0])
    if len(items) == 3:
        threads_per_core = (items[-1].split()[0])
    return pd.Series({
        'Total Cores': total_cores,
        'Chips': chips,
        'Threads Per Core': threads_per_core,
    })
c2017[['Total Cores', 'Chips', 'Threads Per Core']] = c2017['CPU Enabled'].apply(lambda x: parse_cpu_enabled(x))

In [30]:
c2017['Cores Per Chip'] = c2017.apply(lambda item: item['Total Cores'] // item['Chips'], axis=1)

In [38]:
def parse_cpu_orderable(info):
    info = info.split(';')[0]
    nums = [int(num) for num in re.findall(r'\d+', info)]
    return max(nums)
c2017['Max Chips'] = c2017['CPU Orderable'].apply(lambda x: parse_cpu_orderable(x))

In [53]:
c2017['File System'] = c2017['File System'].apply(lambda x: x.lower())

In [73]:
re.findall(r'[(].*?[)]', '256 GB (7 x 16 GB 1Rx4 PC4-3200AA-R; 9 x 16 GB 2Rx8 PC4-3200AA-R)')

['(7 x 16 GB 1Rx4 PC4-3200AA-R; 9 x 16 GB 2Rx8 PC4-3200AA-R)']

In [77]:
def parse_memory(info):
    total_memory_amount = int(info.split()[0])
    unit = info.split()[1]
    if 'T' in unit:
        total_memory_amount *= 1024
    items = re.findall(r'\d+ x', info)
    nums = [int(item.split()[0]) for item in items]
    memory_num = sum(nums)
    memory_detail = re.findall(r'[(].*?[)]', info)[0][1:-1]
    return pd.Series({
        'Total Memory Amount': total_memory_amount,
        'Memory Number': memory_num,
        'Memory Detail': memory_detail,
    })
c2017[['Total Memory Amount', 'Memory Number', 'Memory Detail']] = c2017['Memory'].apply(lambda x: parse_memory(x))  

In [81]:
c2017['Memory Amount'] = c2017.apply(lambda item: item['Total Memory Amount'] // item['Memory Number'], axis=1)

In [110]:
re.findall(r'\d+\.?\d*[ ]*[xz]', '1.01TB  2.01 x 2 z')
# re.findall(r'\d+\.?\d*', '1.01TB  2.01 x 1T')

['2.01 x', '2 z']

In [114]:
def parse_storage(info):
    items = re.findall(r'\d+[ ]*x', info)
    storage_num = 1
    if len(items) != 0:
        storage_num = items[0][:-1].strip()
    storage_num = int(storage_num)
    items = re.findall(r'\d+[.]?\d*[ ]*TB?', info)
    if len(items):
        storage_size = float(re.findall(r'\d+\.?\d*', items[0])[0])
        storage_size = storage_size * 1024
    else:
        items = re.findall(r'\d+[.]*\d*[ ]*GB?', info)
        storage_size = float(re.findall(r'\d+\.?\d*', items[0])[0])
    if 'SSD' in info.upper():
        storage_type = 'SSD'
    elif 'HDD' in info.upper():
        storage_type = 'HDD'
    elif 'ramfs' in info.lower():
        storage_type = 'ramfs'
    elif 'tmpfs' in info.lower():
        storage_type = 'tmpfs'
    elif 'zfs' in info.lower():
        storage_type = 'zfs'
    else:
        storage_type = 'SSD'
    return pd.Series({
        'Storage Number': storage_num,
        'Storage Size': storage_size,
        'Storage Type': storage_type
    })

c2017[['Storage Number', 'Storage Size', 'Storage Type']] = c2017['Storage'].apply(lambda x: parse_storage(x))

In [115]:
c2017['Total Storage Size'] = c2017['Storage Number'] * c2017['Storage Size']

In [118]:
c2017.columns

Index(['Suite', 'HW Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'Max MHz', 'CPU MHz', 'CPU Enabled',
       'CPU Orderable', 'L1', 'L2', 'L3', 'Memory', 'Storage', 'OS',
       'File System', 'URL Suffix', 'CPU Vendor', 'Total Cores', 'Chips',
       'Threads Per Core', 'Cores Per Chip', 'Max Chips', 'L1I', 'L1D',
       'Total Memory Amount', 'Memory Number', 'Memory Detail',
       'Memory Amount', 'Storage Number', 'Storage Size', 'Storage Type',
       'Total Storage Size'],
      dtype='object')

In [57]:
cpu2006_columns = [
    'Suite', 'Hardware Vendor', 'System Name','Baseline',
    'Test Date', 'HW Avail', 'CPU Name', 'CPU Characteristics',
    'CPU MHz', 'CPU(s) enabled', 'CPU(s) orderable', 'Primary Cache',
    'Secondary Cache', 'L3 Cache', 'Memory', 'Disk Subsystem',
    'OS', 'File System', 'URL Suffix',
]
c2006_rfp = c2006_rfp[cpu2006_columns]
c2006_sfp = c2006_sfp[cpu2006_columns]
c2006_rint = c2006_rint[cpu2006_columns]
c2006_sint = c2006_sint[cpu2006_columns]

In [58]:
c2006 = pd.concat([c2006_rfp, c2006_sfp, c2006_rint, c2006_sint]).reset_index(drop=True)

In [68]:
c2006_rename_dict = {
    'Hardware Vendor': 'HW Vendor',
    'CPU(s) enabled': 'CPU Enabled',
    'CPU(s) orderable': 'CPU Orderable',
    'Primary Cache': 'L1',
    'Secondary Cache': 'L2',
    'L3 Cache': 'L3',
    'Disk Subsystem': 'Storage',
}
c2006.rename(columns=c2006_rename_dict, inplace=True)

In [69]:
c2006.columns

Index(['Suite', 'HW Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'CPU Characteristics', 'CPU MHz', 'CPU Enabled',
       'CPU Orderable', 'L1', 'L2', 'L3', 'Memory', 'Storage', 'OS',
       'File System', 'URL Suffix'],
      dtype='object')

In [9]:
jbb2015_columns = [
    'Suite', 'Vendor', 'System Name', 'max_jOPS', 'cirtical_jOPS',
    'Test date', 'Hardware Availability',
    'Total Systems', 'Total Nodes', 'Nodes Per System', 'Total Chips', 'Total Cores',
    'Total Threads', 'Total Memory Amount (GB)', 
    'CPU Name', 'CPU Characteristics', 'Number of Systems', 'Chips Per System',
    'Cores Per System', 'Cores Per Chip', 'Threads Per System',
    'Threads Per Core', 'CPU Frequency (MHz)', 'Primary Cache',
    'Secondary Cache', 'Tertiary Cache',
    'Disk', 'File System', 'Memory Amount (GB)', '# and size of DIMM(s)',
    'Memory Details', 'OS Name', 'OS Vendor', 'OS Version', 'JVM Name', 'JVM Vendor', 'JVM Version',
    'URL Suffix'
]
j2015_com = j2015_com[jbb2015_columns]
j2015_mul = j2015_mul[jbb2015_columns]
j2015_dis = j2015_dis[jbb2015_columns]

In [10]:
j2015 = pd.concat([j2015_com, j2015_mul, j2015_dis]).reset_index(drop=True)

In [ ]:
j2015_rename_dict = {
    'Vendor': 'HW Vendor',
    'Test date': 'Test Date',
    'Hardware Availability': 'HW Avail',
    'CPU Frequency (MHz)': 'CPU MHz',
    'Primary Cache': 'L1',
    'Secondary Cache': 'L2',
    'Tertiary Cache': 'L3',
    'Disk': 'Storage',
}

In [11]:
jvm2008_columns = [
    'Suite', 'HW vendor', 'HW model', 'Result',
    'Test date:', 'HW available', 'CPU name', 'CPU vendor',
    'CPU frequency', '# of logical cpus', '# of chips', '# of cores',
    'Cores per chip', 'Threads per core', 'Threading enabled',
    'Primary cache', 'Secondary cache',
    'Memory size', 'Memory details',
    'OS name', 'Filesystem', 'JVM name', 'JVM version',
    'JVM Vendor', 'URL Suffix'
]
jvm2008 = jvm2008[jvm2008_columns]

In [16]:
jvm2008

,Suite,HW vendor,HW model,Result,Test date:,HW available,CPU name,CPU vendor,CPU frequency,# of logical cpus,...,Secondary cache,Memory size,Memory details,OS name,Filesystem,JVM name,JVM version,JVM available,JVM Vendor,URL Suffix
0,SPECjvm2008,Sugon,Sugon I840-G25(Intel Xeon E7-4830 v2),897.23,Thu Aug 20 04:16:03 GMT-05:00 2015,2014.3,Intel Xeon E7-4830 v2(Intel Turbo Boost Techno...,Intel,2200,40,...,256 KB,256 GB,32 x 8 GB 1Rx4 PC3L-12800R-11,NeoKylin Linux Advanced Server release 6.5 (Be...,ext4,OpenJDK Java 1.7.0 64-bit Server VM,24.45-b08 mixed mode,2013.11,OpenJDK community,res2015q3/jvm2008-20150823-00019.html
1,SPECjvm2008,Sugon,Sugon I620-G20(Intel Xeon E5-2660 v3),853.15,Thu Dec 25 19:18:18 CST 2014,2014.09,Intel Xeon E5-2660 v3(Intel Turbo Boost Techno...,Intel,2600,40,...,256 KB,256 GB,16 x 16 GB 2Rx4 PC4-2133P-R,Red Hat Enterprise Linux 6.5,NaN,OpenJDK 64-Bit Server VM,24.45-b08 mixed mode,2013.11,"Red Hat, Inc.",res2015q1/jvm2008-20150120-00018.html
2,SPECjvm2008,Huawei Technologies Co,RH 2285,335.78,Thu Dec 29 13:52:35 CST 2011,2010,Intel Xeon E5645 (Intel Turbo Boost Technology...,Intel,2.4 GHz,"24 (2 chips, 6 cores/chip, 2 threads/core)",...,"256KB(I+D) on chip, per core",48GB,12*4GB DDR3-1333 DIMM,SUSE Linux Enterprise Server 11 SP1,ext3,Java Hotspot(TM) 64-Bit Server VM on Linux,Java(TM) SE Runtime Environment (build 1.7.0_0...,2011.10,Oracle Corporation,res2012q1/jvm2008-20111230-00013.html
3,SPECjvm2008,Oracle Corporation,Netra SPARC T4-2,454.52,Sun Jan 08 12:35:24 EST 2012,Jan-2012,SPARC T4,Oracle Corporation,2848,128,...,128 KB (I+D) per core,262144,32x 8 GB DDR3-1066 DIMM,Solaris 11 11/11,ZFS,Java HotSpot(TM) 64-Bit Server VM,22.0-b10 mixed mode,Dec-2011,Oracle Corporation,res2012q1/jvm2008-20120110-00014.html
4,SPECjvm2008,Oracle Corporation,SPARC T4-2,454.25,Sat Oct 29 18:23:09 EDT 2011,Oct-2011,SPARC T4,Oracle Corporation,2848,128,...,128 KB (I+D) per core,262144,32x 8 GB DDR3-1066 DIMM,Solaris 11 11/11,ZFS,Java HotSpot(TM) 64-Bit Server VM,22.0-b08 mixed mode,Dec-2011,Oracle Corporation,res2011q4/jvm2008-20111101-00011.html
5,SPECjvm2008,Oracle Corporation,SPARC T3-2,320.52,Thu Sep 16 02:23:05 EDT 2010,Dec-2010,SPARC T3,Oracle Corporation,1.65 GHz,256,...,6 MB per chip,256 GB,32x 8 GB DDR3-1066 DIMM,Solaris 10 9/10,UFS,Java HotSpot(TM) 64-Bit Server VM on Solaris,1.6.0_21 Performance Release,Sep-2010,Oracle Corporation,res2010q4/jvm2008-20100920-00010.html
6,SPECjvm2008,Apple Inc.,iMac (Early 2009),51.60,Fri Oct 02 15:05:27 PDT 2009,Mar-2009,Intel Core 2 Duo CPU E8335,Intel,2930,2,...,6MB (I+D),4096MB,2 x 2GB DDR3 1067MHz,Mac OS X 10.6.1 (10B504),HFS+,Java HotSpot(TM) 64-Bit Server VM,14.1-b02-90 mixed mode,Sep-2009,Apple Inc.,res2009q4/jvm2008-20091021-00008.html
7,SPECjvm2008,Apple Inc.,iMac (Early 2009),37.37,Mon Oct 05 10:28:48 PDT 2009,Mar-2009,Intel Core 2 Duo CPU E8335,Intel,2930,2,...,6MB (I+D),4096MB,2 x 2GB DDR3 1067MHz,Mac OS X 10.6.1 (10B504),HFS+,Java HotSpot(TM) Client VM,14.1-b02-90 mixed mode,Sep-2009,Apple Inc.,res2009q4/jvm2008-20091021-00007.html
8,SPECjvm2008,Apple Inc.,iMac (Early 2009),26.37,Fri Oct 02 11:19:32 PDT 2009,Mar-2009,Intel Core 2 Duo CPU E8335,Intel,2930,2,...,6MB (I+D),4096MB,2 x 2GB DDR3 1067MHz,Mac OS X 10.5.8 (9L31a),HFS+,Java HotSpot(TM) Client VM,1.5.0_20-141 mixed mode,Sep-2009,Apple Inc.,res2009q4/jvm2008-20091021-00006.html
9,SPECjvm2008,Sun Microsystems Inc.,Sun Blade X6270,317.13,Wed Mar 25 12:12:12 EDT 2009,Apr-2009,Intel Xeon X5570 (Intel Turbo Boost Technology...,Intel,2.93 GHz,"16 (2 chips, 4 cores/chip, 2 threads/core)",...,256 KB I+D on chip per core,48GB,12x 4GB DDR3-1066 DIMM,OpenSolaris 2008.11,ZFS,Java HotSpot(TM) 64-Bit Server VM,1.6.0_14 Performance Release,Jun-2009,Sun Microsystems Inc.,res2009q2/jvm2008-20090330-00005.html


In [14]:
ssj2008_columns = [
    'Suite', 'Hardware Vendor', 'Model', 'Benchmark',
    'Test Date', 'Hardware Availability', 'Test Method',
    'CPU Name', 'CPU Characteristics', 'CPU Frequency (MHz)', 'CPU(s) Enabled',
    'Hardware Threads', 'CPU(s) Orderable', 'Primary Cache',
    'Secondary Cache', 'Tertiary Cache', 'Memory Amount (GB)', '# and size of DIMM', 'Memory Details',
    'Disk Drive', 'Operating System (OS)', 'OS Version', 'Filesystem', 'JVM Vendor', 'JVM Version',
    'URL Suffix'
]
ssj2008 = ssj2008[ssj2008_columns]

In [15]:
ssj2008

,Suite,Hardware Vendor,Model,Benchmark,Test Date,Hardware Availability,Test Method,CPU Name,CPU Characteristics,CPU Frequency (MHz),...,Memory Amount (GB),# and size of DIMM,Memory Details,Disk Drive,Operating System (OS),OS Version,Filesystem,JVM Vendor,JVM Version,URL Suffix
0,SPECpower_ssj2008,ASUSTeK Computer Inc.,RS720-E9/RS8,"12,727","Sep 26, 2018",Sep-2018,Single Node,Intel Xeon Platinum 8180,"28-Core, 2.5 GHz, 38.5MB L3 Cache(Intel Turbo ...",2500,...,192.0,12 x 16 GB,12 x 16GB 2Rx8 PC4-2666V ECC RDIMM;slots DIMM_...,1 x 480GB Intel 535 SATA 2.5 SSD,Microsoft Windows Server 2012 R2 Datacenter,Version 6.3 (Build 9600),NTFS,Oracle Corporation,Oracle Java HotSpot(TM) 64-Bit Server VM (buil...,res2018q4/power_ssj2008-20181009-00863.html
1,SPECpower_ssj2008,Acer Incorporated,"Acer AR385 F1 (AMD Opteron 6174, 2.20 GHz)","2,424","Oct 20, 2010",Oct-2010,Single Node,AMD Opteron 6174,"Twelve Core, 2.20 GHz, 12 MB L3 Cache",2200,...,16.0,8 x 2048 MB,"2GB 2Rx8 PC3-10600E; slots 1A, 2A, 3A and 4A p...","1 x 64GB SSD 2.5"" SATA, Acer P/N:KF.0640N.001","Windows Server 2008, Enterprise Edition",R2,NTFS,IBM Corporation,"IBM J9 VM (build 2.4, JRE 1.6.0 IBM J9 2.4 Win...",res2010q4/power_ssj2008-20101021-00306.html
2,SPECpower_ssj2008,Acer Incorporated,Acer AW2000h-AW175h F1,"2,474","Oct 5, 2010",Aug-2010,Multi Node,AMD Opteron 6174,"12-Core, 2.20 GHz, 12 MB L3 cache",2200,...,16.0,8 x 2048 MB,"2GB 1Rx8 PC3-10600E; slots 1A, 2A, 3A and 4A p...","1 x 64GB SSD 2.5"" SATA, Acer P/N:KF.0640N.001","Windows Server 2008, Enterprise Edition",R2,NTFS,IBM Corporation,"IBM J9 VM (build 2.4, JRE 1.6.0 IBM J9 2.4 Win...",res2010q4/power_ssj2008-20101007-00299.html
3,SPECpower_ssj2008,Acer Incorporated,Gateway GW2000h-GW175h F1,"2,481","Oct 5, 2010",Aug-2010,Multi Node,AMD Opteron 6174,"12-Core, 2.20 GHz, 12 MB L3 cache",2200,...,16.0,8 x 2048 MB,"2GB 1Rx8 PC3-10600E; slots 1A, 2A, 3A and 4A p...","1 x 64GB SSD 2.5"" SATA, Acer P/N:KF.0640N.001","Windows Server 2008, Enterprise Edition",R2,NTFS,IBM Corporation,"IBM J9 VM (build 2.4, JRE 1.6.0 IBM J9 2.4 Win...",res2010q4/power_ssj2008-20101007-00298.html
4,SPECpower_ssj2008,Acer Incorporated,"Acer AT115 F1 (AMD Opteron 4164EE, 1.80 GHz)","1,980","Sep 29, 2010",Oct-2010,Single Node,AMD Opteron 4164EE,"Six-Core, 1.8GHz, 6MB L3 Cache",1800,...,8.0,2 x 4096 MB,"4GB, PC3-1066R- ECC, slots 1A, 1B populated","1 x 64 GB SSD 2.5"" SATA (Intel SSDSA2SH064G1GC...","Windows Server 2008, Enterprise Edition",R2,NTFS,IBM Corporation,"IBM J9 VM <build 2.4, JRE 1.6.0 IBM J9 2.4 Win...",res2010q4/power_ssj2008-20101019-00305.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,SPECpower_ssj2008,ASUSTeK Computer Inc.,RS500A-E10-PS4,"21,168","Nov 22, 2019",Oct-2019,Single Node,AMD EPYC 7742 2.25Ghz,"64-Core, 2.25GHz, 256MB L3 Cache",2250,...,128.0,8 x 16 GB,8 x 16GB 2Rx8 PC4-2933Y ECC RDIMM; slots DIMM_...,1 x 960GB Intel DC S4510 SATA 2.5 SSD P/N: 03B...,SUSE Linux Enterprise Server 12 SP4,4.12.14-94.41-default,btrfs,Oracle,Oracle Java HotSpot(TM) 64-Bit Server VM (buil...,res2020q1/power_ssj2008-20191125-01012.html
774,SPECpower_ssj2008,ASUSTeK Computer Inc.,RS500A-E10-PS4,"20,908","Nov 21, 2019",Oct-2019,Single Node,AMD EPYC 7742 2.25Ghz,"64-Core, 2.25GHz, 256MB L3 Cache",2250,...,128.0,8 x 16 GB,8 x 16GB 2Rx8 PC4-2933Y ECC RDIMM; slots DIMM_...,1 x 960GB Intel DC S4510 SATA 2.5 SSD P/N: 03B...,Microsoft Windows Server 2019 Datacenter,Version 1809 (OS Build 17763.720),NTFS,Oracle,Java HotSpot(TM) 64-Bit Server VM (build 12.0....,res2020q1/power_ssj2008-20191125-01011.html
775,SPECpower_ssj2008,ASUSTeK Computer Inc.,RS720-E9-RS8,"14,274","May 9, 2019",Apr-2019,Single Node,Intel Xeon Platinum 8280L,"28-Core, 2.7GHz, 38.5MB L3 Cache",2700,...,192.0,12 x 16 GB,12 x 16GB 2Rx8 PC4-2933Y ECC RDIMM; slots DIMM...,1 x 960GB Intel DC S3520 SATA 2.5 SSD P/N: 03B...,Microsoft Windows Server 2012 R2 Datacenter,Version 6.3 (Build 9600),NTFS,Oracle Corporation,Oracle Java HotSpot(TM) 64-Bi

In [12]:
ssj2008.columns

Index(['Suite', 'Test Sponsor', 'SPEC License #', 'Test Method', 'Tested By',
       'Test Date', 'Hardware Availability', 'Software Availability',
       'Publication', 'ssj_ops @ 100% of target load',
       'Average watts @ 100% of target load',
       'Performance/power @ 100% of target load',
       'ssj_ops @ 90% of target load', 'Average watts @ 90% of target load',
       'Performance/power @ 90% of target load',
       'ssj_ops @ 80% of target load', 'Average watts @ 80% of target load',
       'Performance/power @ 80% of target load',
       'ssj_ops @ 70% of target load', 'Average watts @ 70% of target load',
       'Performance/power @ 70% of target load',
       'ssj_ops @ 60% of target load', 'Average watts @ 60% of target load',
       'Performance/power @ 60% of target load',
       'ssj_ops @ 50% of target load', 'Average watts @ 50% of target load',
       'Performance/power @ 50% of target load',
       'ssj_ops @ 40% of target load', 'Average watts @ 40% of target l

In [49]:
jvm2008.columns

Index(['Suite', 'Result', 'Submitter', 'SPEC license', 'Test date:',
       'HW vendor', 'HW model', 'HW available', 'CPU vendor', 'CPU name',
       'CPU frequency', '# of logical cpus', '# of chips', '# of cores',
       'Cores per chip', 'Threads per core', 'Threading enabled',
       'HW address bits', 'Primary cache', 'Secondary cache', 'Other cache',
       'Memory size', 'Memory details', 'Other HW details', 'OS name',
       'OS available', 'Filesystem', 'JVM name', 'JVM version',
       'JVM available', 'JVM Vendor', 'URL Suffix'],
      dtype='object')

In [45]:
c2006_rfp.columns

Index(['Suite', 'Hardware Vendor', 'System Name', 'Metric', 'Baseline',
       'License', 'Test Sponsor', 'Tested By', 'Test Date', 'HW Avail',
       'SW Avail', 'CPU Name', 'CPU Characteristics', 'CPU MHz', 'FPU',
       'CPU(s) enabled', 'CPU(s) orderable', 'Primary Cache',
       'Secondary Cache', 'L3 Cache', 'Other Cache', 'Memory',
       'Disk Subsystem', 'Other Hardware', 'OS', 'Compiler', 'Parallel',
       'File System', 'System State', 'Base Pointers', 'Peak Pointers',
       'Other Software', '410.bwaves', '416.gamess', '433.milc', '434.zeusmp',
       '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd',
       '447.dealII', '450.soplex', '453.povray', '454.calculix',
       '459.GemsFDTD', '465.tonto', '470.lbm', '481.wrf', '482.sphinx3',
       'Base Copies', 'URL Suffix'],
      dtype='object')

In [8]:
c2017_rfp['CPU Name'].unique()

array(['Intel Xeon Platinum 8360Y', 'Intel Xeon Gold 6338N',
       'Intel Xeon Platinum 8276L', 'Intel Xeon Gold 6330N',
       'Intel Xeon Platinum 8352Y', 'Intel Xeon Gold 6230R',
       'Intel Xeon Gold 6338', 'Intel Xeon Gold 6326',
       'Intel Xeon Gold 6334', 'Intel Xeon Gold 6336Y',
       'Intel Xeon Gold 6342', 'Intel Xeon Silver 4316',
       'Intel Xeon Silver 4314', 'Intel Xeon Gold 5320',
       'Intel Xeon Silver 4310', 'Intel Xeon Platinum 8376HL',
       'Intel Xeon Platinum 8376H', 'Intel Xeon Platinum 8280',
       'Intel Xeon Gold 6348H', 'Intel Xeon Gold 6312U',
       'Intel Xeon Platinum 8380HL', 'Intel Xeon Platinum 8380',
       'Intel Xeon Gold 6248', 'Intel Xeon Gold 6248R', 'IBM POWER8',
       'Intel Xeon Gold 6354', 'AMD EPYC 7773X', 'AMD EPYC 7573X',
       'AMD EPYC 7473X', 'AMD EPYC 7373X', 'Intel Xeon Gold 5317',
       'Intel Xeon E-2378G', 'AMD EPYC 7543', 'AMD EPYC 7313',
       'AMD EPYC 7513', 'AMD EPYC 7313P', 'AMD EPYC 7702',
       'Intel Xeo

In [42]:
jvm2008[jvm2008.isnull().any(axis=1)].iloc[-1]

Suite                                                      SPECjvm2008
Result                                                          260.08
Submitter                                       Sun Microsystems, Inc.
SPEC license                                                         6
Test date:                                Wed Jun 04 04:54:27 EDT 2008
HW vendor                                       Sun Microsystems, Inc.
HW model                                                Sun Fire X4450
HW available                                                  Mar-2008
CPU vendor                                                       Intel
CPU name              Intel Xeon X7350 Quad Core (1066 MHz system bus)
CPU frequency                                                     2933
# of logical cpus                                                   16
# of chips                                                           4
# of cores                                                          16
Cores 

In [29]:
c2017_rfp = c2017_rfp[c2017_rfp.notnull().any(axis=1)]

,Suite,Hardware Vendor,System Name,Metric,Baseline,License,Test Sponsor,Tested By,Test Date,HW Avail,...,519.lbm_r,521.wrf_r,526.blender_r,527.cam4_r,538.imagick_r,544.nab_r,549.fotonik3d_r,554.roms_r,Base Copies,URL Suffix
0,CFP2017_rate,xFusion,xFusion 5288 V6 (Intel Xeon Platinum 8360Y),SPECrate®2017_fp_base,455.0,6488,xFusion,xFusion,Apr-2022,Apr-2021,...,279.0,356.0,532.0,513.0,1450.0,912.0,234.0,162.0,144,res2022q2/cpu2017-20220411-31451.html
1,CFP2017_rate,xFusion,xFusion 1288H V6 (Intel Xeon Gold 6338N),SPECrate®2017_fp_base,380.0,6488,xFusion,xFusion,Apr-2022,Apr-2021,...,242.0,311.0,424.0,419.0,1170.0,729.0,207.0,142.0,128,res2022q2/cpu2017-20220410-31429.html
2,CFP2017_rate,xFusion,xFusion 2288H V6 (Intel Xeon Platinum 8360Y),SPECrate®2017_fp_base,455.0,6488,xFusion,xFusion,Apr-2022,Apr-2021,...,279.0,357.0,532.0,512.0,1440.0,914.0,233.0,163.0,144,res2022q2/cpu2017-20220411-31447.html
3,CFP2017_rate,xFusion,xFusion 1288H V6 (Intel Xeon Platinum 8360Y),SPECrate®2017_fp_base,455.0,6488,xFusion,xFusion,Apr-2022,Apr-2021,...,279.0,357.0,531.0,510.0,1450.0,913.0,234.0,163.0,144,res2022q2/cpu2017-20220411-31449.html
4,CFP2017_rate,xFusion,xFusion 5288 V6 (Intel Xeon Gold 6338N),SPECrate®2017_fp_base,380.0,6488,xFusion,xFusion,Apr-2022,Apr-2021,...,243.0,312.0,424.0,417.0,1170.0,725.0,207.0,143.0,128,res2022q2/cpu2017-20220410-31431.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7616,CFP2017_rate,Huawei,Huawei Kunlun 9008 V5 (Intel Xeon Platinum 8280),SPECrate2017_fp_base,859.0,3175,Huawei,Huawei,Mar-2019,Apr-2019,...,401.0,730.0,1110.0,1060.0,2420.0,1800.0,518.0,315.0,336,res2019q2/cpu2017-20190319-11484.html
7617,CFP2017_rate,Huawei,Huawei Kunlun 9008 V5 (Intel Xeon Platinum 8280),SPECrate2017_fp_base,1160.0,3175,Huawei,Huawei,Mar-2019,Apr-2019,...,534.0,972.0,1470.0,1450.0,3410.0,2400.0,697.0,426.0,448,res2019q2/cpu2017-20190319-11487.html
7618,CFP2017_rate,Huawei,Huawei 2288H V5 (Intel Xeon Gold 5220),SPECrate2017_fp_base,190.0,3175,Huawei,Huawei,Mar-2019,Apr-2019,...,109.0,193.0,209.0,211.0,449.0,317.0,151.0,87.3,72,res2019q2/cpu2017-20190404-11699.html
7619,CFP2017_rate,Huawei,Huawei 2288H V5 (Intel Xeon Platinum 8268),SPECrate2017_fp_base,266.0,3175,Huawei,Huawei,Mar-2019,Apr-2019,...,132.0,246.0,328.0,326.0,711.0,517.0,178.0,109.0,96,res2019q2/cpu2017-20190404-11702.html


In [22]:
c2017_rfp.loc[c2017_rfp['Max MHz'].isnull(), 'Max MHz']

7201   NaN
7230   NaN
Name: Max MHz, dtype: float64

In [28]:
c2017_sint.loc[c2017_sint['Max MHz'].isnull(), :]

,Suite,Hardware Vendor,System Name,Metric,Baseline,License,Test Sponsor,Tested By,Test Date,HW Avail,...,605.mcf_s,620.omnetpp_s,623.xalancbmk_s,625.x264_s,631.deepsjeng_s,641.leela_s,648.exchange2_s,657.xz_s,Base Threads,URL Suffix
5798,CINT2017_speed,Cisco Systems,"Cisco UCS B200 M5 (Intel Xeon Bronze 3104, 1.7...",SPECspeed®2017_int_base,4.11,9019,Cisco Systems,Cisco Systems,Jun-2018,Aug-2017,...,5.88,3.05,4.49,4.89,2.54,2.01,6.07,8.36,12,res2018q2/cpu2017-20180612-06907.html
5824,CINT2017_speed,Cisco Systems,"Cisco UCS C220 M5 (Intel Xeon Bronze 3104, 1.7...",SPECspeed®2017_int_base,4.13,9019,Cisco Systems,Cisco Systems,Dec-2017,Aug-2017,...,5.80,3.05,4.44,5.43,2.51,1.99,6.14,8.31,12,res2018q1/cpu2017-20171225-02061.html


In [20]:
c2017_rfp['Baseline'].isnull().values.any()
c2017_rfp.loc[c2017_rfp['Baseline'].isnull(), :]

,Suite,Hardware Vendor,System Name,Metric,Baseline,License,Test Sponsor,Tested By,Test Date,HW Avail,...,519.lbm_r,521.wrf_r,526.blender_r,527.cam4_r,538.imagick_r,544.nab_r,549.fotonik3d_r,554.roms_r,Base Copies,URL Suffix
4509,CFP2017_rate,Hewlett Packard Enterprise (Test Sponsor:...,"ProLiant DL325 Gen10 (2.00 GHz, AMD EPYC 7401)",SPECrate2017_fp_base,NaN,3,HPE,HPE,Jul-2018,Jul-2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2018q3/cpu2017-20180806-08164.html
4515,CFP2017_rate,Hewlett Packard Enterprise (Test Sponsor:...,"ProLiant DL325 Gen10 (2.00 GHz, AMD EPYC 7401)",SPECrate2017_fp_base,NaN,3,HPE,HPE,Jul-2018,Jul-2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2018q3/cpu2017-20180806-08152.html
5791,CFP2017_rate,Dell Inc.,PowerEdge R6515 (redacted),SPECrate®2017_fp_base,NaN,55,Dell Inc.,Dell Inc.,Jan-2020,Apr-2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2020q1/cpu2017-20200119-20585.html
6905,CFP2017_rate,Cisco Systems,"Cisco UCS B480 M5 (Intel Xeon Platinum 8256, 3...",SPECrate®2017_fp_base,NaN,9019,Cisco Systems,Cisco Systems,Nov-2019,Apr-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2019q4/cpu2017-20191112-19830.html
6947,CFP2017_rate,Cisco Systems,"Cisco UCS C220 M5 (Intel Xeon Platinum 8256, 3...",SPECrate®2017_fp_base,NaN,9019,Cisco Systems,Cisco Systems,Oct-2019,Apr-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2019q4/cpu2017-20191014-19009.html
6957,CFP2017_rate,Cisco Systems,"Cisco UCS B200 M5 (Intel Xeon Platinum 8256, 3...",SPECrate®2017_fp_base,NaN,9019,Cisco Systems,Cisco Systems,Sep-2019,Apr-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2019q4/cpu2017-20191001-18918.html
6977,CFP2017_rate,Cisco Systems,"Cisco UCS C220 M5 (Intel Xeon Platinum 8253, 2...",SPECrate®2017_fp_base,NaN,9019,Cisco Systems,Cisco Systems,Sep-2019,Apr-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2019q4/cpu2017-20190930-18786.html
7016,CFP2017_rate,Cisco Systems,"Cisco UCS C240 M5 (Intel Xeon Platinum 8256, 3...",SPECrate®2017_fp_base,NaN,9019,Cisco Systems,Cisco Systems,Aug-2019,Apr-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2019q3/cpu2017-20190903-17776.html
7018,CFP2017_rate,Cisco Systems,"Cisco UCS C240 M5 (Intel Xeon Platinum 8253, 2...",SPECrate®2017_fp_base,NaN,9019,Cisco Systems,Cisco Systems,Aug-2019,Apr-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2019q3/cpu2017-20190903-17770.html
7025,CFP2017_rate,Cisco Systems,"Cisco UCS B200 M5 (Intel Xeon Platinum 8253, 2...",SPECrate®2017_fp_base,NaN,9019,Cisco Systems,Cisco Systems,Jul-2019,Apr-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,res2019q3/cpu2017-20190903-17742.html


In [14]:
c2017_rfp.iloc[7201]

Suite                                                   CFP2017_rate
Hardware Vendor                                        Cisco Systems
System Name        Cisco UCS B200 M5 (Intel Xeon Bronze 3104, 1.7...
Metric                                         SPECrate®2017_fp_base
Baseline                                                        45.6
License                                                         9019
Test Sponsor                                           Cisco Systems
Tested By                                              Cisco Systems
Test Date                                                   Jun-2018
HW Avail                                                    Aug-2017
SW Avail                                                    Mar-2018
CPU Name                                      Intel Xeon Bronze 3104
Max MHz                                                          NaN
Nominal                                                         1700
Enabled                           